In [ ]:
pip install streamlit boto3

In [ ]:
%%writefile app.py
import streamlit as st
import boto3
import json

# Initialize AWS Lambda client with region
client = boto3.client('lambda', region_name='us-east-1',
                      aws_access_key_id='ENTER ACCESS KEY',
                      aws_secret_access_key='ENTER SECRET KEY')  # Replace with your AWS credentials

# Function to send post to AWS Lambda
def send_post_to_lambda(post_text):
    response = client.invoke(
        FunctionName='NAME',  # Replace with your actual Lambda function name
        InvocationType='RequestResponse',
        Payload=json.dumps({"post_text": post_text})
    )
    return response

# Streamlit App Title
st.title("Social Media Hashtag Trend Analyzer")

# Create a form for post submission
with st.form("post_form"):
    # Text area for the user to write the post
    post_text = st.text_area("Write your post here...")

    # Submit button
    submit_button = st.form_submit_button(label="Post")

# When the user clicks the submit button
if submit_button:
    # Check if the post text is not empty
    if post_text.strip():
        # Send the post to AWS Lambda
        response = send_post_to_lambda(post_text)

        # Display a simple "Post Sent" message
        st.success("Post Sent.")
    else:
        # If no text is entered, show a warning message
        st.warning("Please enter some text before submitting.")

# Function to fetch trending hashtags from another Lambda function



# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')  # Update with your region if needed
table = dynamodb.Table('TABLENAME')  # Replace with your table name

def fetch_trending_hashtags():
    try:
        # Scan the table to get all items with the 'hashtags' attribute
        response = table.scan()
        items = response.get('Items', [])
        
        # Extract hashtags
        hashtags = [item['hashtags'] for item in items if 'hashtags' in item]
        
        return hashtags
    except Exception as e:
        st.error(f"Error fetching trending hashtags: {str(e)}")
        return []

# Add a button to fetch trending hashtags
if st.button("Show Trending Hashtags"):
    trending_hashtags = fetch_trending_hashtags()
    
    # Check if there are any trending hashtags
    if trending_hashtags:
        # Convert hashtags to a list of dictionaries for table display
        hashtags_data = [{'Hashtag': hashtag} for hashtag in trending_hashtags]

        # Display trending hashtags in a table format
        st.write("### Trending Hashtags:")
        st.table(hashtags_data)  # or use st.dataframe(hashtags_data)
    else:
        st.write("No trending hashtags found.")


In [ ]:
import subprocess
import time
from IPython.display import IFrame

# Function to run Streamlit app
def run_streamlit():
    subprocess.Popen(["streamlit", "run", "app.py"])

# Run Streamlit app
run_streamlit()

# Allow time for the Streamlit server to start
time.sleep(10)  # Adjust the sleep time as needed

# Define the URL of your Streamlit app
public_url = 'http://localhost:8501'  # Replace with the appropriate port if different

# Display the Streamlit app in an iframe
display(IFrame(src=public_url, width=800, height=600))
